<a href="https://colab.research.google.com/github/chathumal93/ADB-EARR-T4/blob/main/helper_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# function for CSV to Dataframe conversion (PSI-result)
def psi_csv2df(csv_file_path):
  """
  This function converts the GEP PSI products in to a dataframe
  """
  # Read PSI data in csv format into a dataframe
  data = pd.read_csv(csv_file_path,sep=';',skiprows=16)
  
  # Image list (dates) in the csv
  slv_img_list = list(data.columns)[6:]
                              
  # Renaming the columns
  column_names_01 = ['ID','Lat','Lon','V_LOS','V_STD','INC_ANG']

  # Image name list with date
  column_names_02 = []

  # Re-arranging the column names (This is based on the EARR project geo-portal visualization requirement)
  # Getting the date tag of images
  for datetime_tag in range(len(slv_img_list)):
      column_names_02.append('DL'+ str(slv_img_list[datetime_tag]))

  # Final Column names for the dataframe    
  column_all = column_names_01 + column_names_02

  # Final Dataframe creation with the arranged data and column names
  data.columns = column_all

  return data

In [ ]:
# function for CSV to Dataframe conversion (PSBAS-result)
def sbas_csv2df(csv_file_path):
  data = pd.read_csv(csv_file_path,skiprows=43)
  # Column Adjusting
  column_names_01 = ['ID','Lat','Lon','Topo','V_LOS','COH','cosN','cosE','cosU','TS']

  # Getting the image list (Frist image as the base image)
  img_list = list(pd.read_csv(csv_file_path, nrows=1, skiprows=39))
  slv_img_list = img_list[1:]
  # Slave image name list
  column_names_02 = []

  # Getting the data tag of slave images
  for datetime_tag in range(len(slv_img_list)):
      date_tag = slv_img_list[datetime_tag].split('T')[0].split(' ')[1]
      column_names_02.append('DL'+ date_tag.split('-')[0]+''+date_tag.split('-')[1]+''+date_tag.split('-')[2])    
  column_all = column_names_01 + column_names_02
  data.columns = column_all

  #Conversion for the mm scale
  data['V_LOS'] = np.array(data['V_LOS'])*10
  #Conversion of displacement to mm scale
  for col in column_names_02:
    data[col] = np.array(data[col])*10

  return data

In [ ]:
def df2gdf_utm(df):
  """
  This function converts the data frame  in to a geodataframe based on WGS 84 / UTM zone 50S
  df: dataframe of PSI data (Asc/Des)
  """
  # Accessing the Lat and Long from dataframe for geometry conversion
  geometry = [Point(xy) for xy in zip(df.Lon, df.Lat)]
  gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

  # Coverting gdf from EPSG:4326 to EPSG:32750 – WGS 84 / UTM zone 50S
  # For the stabilty analysis the points needs to be in meter(m) format
  gdf_utm = gdf.to_crs("EPSG:32750")

  return gdf_utm

In [ ]:
def NN_buf_KDTree(New_CO,Old_V,Old_CO,buffer_m,npt):

  """
  This function assigns the nearest neighbour value for a new point based on 
  the difined distance from a old point which has the location and value components.

  New_CO : Coordinates of new points (to assign the interpolated data)
  Old_CO : Coordinates of old points
  Old_V : Values of old points to be interpolated
  buffer_m : Buffer distance in meters
  np : number of closest points needed
  """  
  Tree = KDTree(Old_CO,)

  NN_Buff_Values = []
  for item in range(len(New_CO)):

    dd, ii = Tree.query(New_CO[item], k=npt)

    # [distance , index]
    dd_ii = np.c_[dd,ii]

    if (dd_ii[0,0] <= buffer_m) & (dd_ii[0,0] >= 0):
      intpl_value = Old_V[int(dd_ii[0][1])][0]
      NN_Buff_Values.append(intpl_value)

    else :
      intpl_value = -9999
      NN_Buff_Values.append(intpl_value)

  return NN_Buff_Values